# Setup

In [29]:
# TODO: add annotations describing usage of different modules

from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, KBinsDiscretizer
from sklearn.svm import SVR
import pandas as pd
import numpy as np
# from ydata_profiling import ProfileReport

from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
# import xgboost as xgb
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings

from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

import altair as alt
import random
import warnings

warnings.filterwarnings('ignore')

# set seaborn whitegrid theme
sns.set(style="whitegrid")

from sklearn.inspection import permutation_importance
from random import sample
from itertools import combinations

In [30]:
def get_model_data(original_df, 
                   columns, 
                   test_size_to_use=0.2,
                   drop_null_rows=False,
                   null_imputer_strategy='median', # mean, median, most_frequent
                   use_value_scaler=True,
                   use_smote=False,
                   return_indices=False):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    # add target column (MDD)
    cols_to_use = columns.copy()
    cols_to_use.insert(0, 'MDD')
    #cols_to_use.insert(0, 'SEQN')
    
    df_to_use = original_df[cols_to_use]
    
    if drop_null_rows:
        df_to_use.dropna(inplace=True)
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    indices = np.arange(y.shape[0])
    
    if not drop_null_rows:
        # SimpleImputer() = fill in missing values
        # note imputer may drop columns if no values exist for it
        imputer = SimpleImputer(strategy=null_imputer_strategy)  
        x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    if use_value_scaler:
        trans = RobustScaler()
        x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test, idx_train, idx_test = train_test_split(
        x, 
        y, 
        indices,
        test_size=test_size_to_use, 
        random_state=42
    ) 
    
    # Technique to de-risk from positive class imbalance
    if use_smote:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)
    
    if return_indices:
        return x_train, x_test, y_train, y_test, idx_train, idx_test
    else:
        return x_train, x_test, y_train, y_test
    
    
def get_performance_df(label_actual, label_pred, model_name):
    """
    Function to calculate performance metrics for model.
    Includes precision, recal, F1, & support.
    """
    # create classification report
    result_table = classification_report(label_actual, label_pred, output_dict=True)
    result_table = pd.DataFrame.from_dict(result_table)

    # store for later
    accuracies = result_table['accuracy'][0]
    
    column_key = {
        '0':'Depressed (No)',
         '1':'Depressed (Yes)',
         'accuracy':'accuracy',
         'macro avg':'Macro Avg',
         'weighted avg':'Weighted Avg'
    }

    # rename grouping
    result_table.columns = [column_key.get(key) for key in result_table.columns]

    # create dataframe with 1 row per grouping
    result_table.drop(labels = 'accuracy', axis = 1, inplace=True)
    result_table = result_table.transpose()
    result_table['accuracy'] = [accuracies for i in range(result_table.shape[0])]
    result_table = result_table.reset_index()
    result_table.rename(columns = {'index':'grouping'},inplace=True)
    result_table['model'] = model_name
    result_table = result_table[['model','grouping','precision','recall','f1-score','support','accuracy']]
    return result_table    

In [31]:
cdc_survey = pd.read_csv('../../../data/cdc_nhanes_survey_responses_clean.csv')
print(cdc_survey.shape)

# filter to pregnant moms
cdc_survey_pmom = cdc_survey[cdc_survey['has_been_pregnant'] == 1].reset_index()
print(cdc_survey_pmom.shape)

(35706, 863)
(7741, 864)


## Create subset datasets 
- cdc_survey_pmom_1plus_dep_screener = 1+ dep screener above 0
- cdc_survey_pmom_0dep_screener = 0 dep screener above 0

In [32]:
dep_screener_cols = [
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks'
]

cdc_survey_pmom['num_dep_screener_0'] = (cdc_survey_pmom[dep_screener_cols]==0).sum(axis=1)
cdc_survey_pmom['num_dep_screener_nonzero'] = (cdc_survey_pmom[dep_screener_cols]>0).sum(axis=1)
cdc_survey_pmom['num_dep_screener_null'] = cdc_survey_pmom[dep_screener_cols].isnull().sum(axis=1)
cdc_survey_pmom['weight_lbs_over_height_in_ratio'] = round(cdc_survey_pmom['weight_lbs'] / cdc_survey_pmom['height_in'],1)

cdc_survey_pmom_1plus_dep_screener = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] < 9]
#cdc_survey_pmom_1plus_dep_screener = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_nonzero'] >= 1]
print(cdc_survey_pmom_1plus_dep_screener.shape)

cdc_survey_pmom_0dep_screener = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_0'] >= 9]
#cdc_survey_pmom_0dep_screener = cdc_survey_pmom[cdc_survey_pmom['num_dep_screener_nonzero'] == 0]
print(cdc_survey_pmom_0dep_screener.shape)

(4394, 868)
(3347, 868)


# GNB Classifier (train & predict on cdc_survey_pmom_1plus_dep_screener)

## Current Model

In [33]:
cdc_survey_pmom_1plus_dep_screener['weight_lbs_over_height_in_ratio'] = round(cdc_survey_pmom_1plus_dep_screener['weight_lbs'] / cdc_survey_pmom_1plus_dep_screener['height_in'],1)

gnb_model_features = dep_screener_cols + ['seen_mental_health_professional',
 'times_with_12plus_alc',
 'time_since_last_healthcare',
 'cholesterol_prescription',
 'high_cholesterol',
 'age_in_years',
 'horomones_not_bc',
 'months_since_birth',
 'arthritis',
 'high_bp',
 'regular_periods',
 'moderate_recreation',
 'thyroid_issues',
 'vigorous_recreation',
 'stroke',
 'is_usa_born',
 'asthma',
 'count_days_moderate_recreational_activity',
 'have_health_insurance',
 'num_dep_screener_0',
 'weight_lbs_over_height_in_ratio'
]

In [34]:
gnb_x_train, gnb_x_test, gnb_y_train, gnb_y_test = get_model_data(
    original_df = cdc_survey_pmom_1plus_dep_screener,
    columns = gnb_model_features
)

print(gnb_x_train.shape)
print(gnb_x_test.shape)
print(gnb_y_train.shape)
print(gnb_y_test.shape)

gnb = GaussianNB()
gnb.fit(gnb_x_train, gnb_y_train)
gnb_pred  = gnb.predict(gnb_x_test)
gnb_score = get_performance_df(gnb_y_test, gnb_pred,'Gaussian Naive Bayes')  
gnb_score

(3515, 31)
(879, 31)
(3515,)
(879,)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gaussian Naive Bayes,Depressed (No),0.930757,0.782138,0.850000,739.0,0.767918
1,Gaussian Naive Bayes,Depressed (Yes),0.375969,0.692857,0.487437,140.0,0.767918
2,Gaussian Naive Bayes,Macro Avg,0.653363,0.737498,0.668719,879.0,0.767918
3,Gaussian Naive Bayes,Weighted Avg,0.842395,0.767918,0.792254,879.0,0.767918


## Test Hyperparameters

In [15]:
gnb_param_grid = {'var_smoothing': np.logspace(0,-9, num=100)}
#gnb_param_grid

In [16]:
gnb_grid_search = GridSearchCV(
    GaussianNB(),
    param_grid=gnb_param_grid
)
gnb_grid_search.fit(gnb_x_train, gnb_y_train)

GridSearchCV(estimator=GaussianNB(),
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02, 2.3101297...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])})

In [17]:
print(gnb_grid_search.best_estimator_)

GaussianNB(var_smoothing=1.0)


In [12]:
gnb_hyper = GaussianNB(var_smoothing=1.0)
gnb_hyper.fit(gnb_x_train, gnb_y_train)
gnb_hyper_pred  = gnb_hyper.predict(gnb_x_test)
gnb_hyper_score = get_performance_df(gnb_y_test, gnb_hyper_pred,'Gaussian Naive Bayes')  
gnb_hyper_score

,model,grouping,precision,recall,f1-score,support,accuracy
0,Gaussian Naive Bayes,Depressed (No),0.849537,0.993234,0.915783,739.0,0.846416
1,Gaussian Naive Bayes,Depressed (Yes),0.666667,0.071429,0.129032,140.0,0.846416
2,Gaussian Naive Bayes,Macro Avg,0.758102,0.532331,0.522408,879.0,0.846416
3,Gaussian Naive Bayes,Weighted Avg,0.820411,0.846416,0.790476,879.0,0.846416


# GB Classifier (train & predict on cdc_survey_pmom_1plus_dep_screener)

## Current Model

In [58]:
gb_x_train, gb_x_test, gb_y_train, gb_y_test = get_model_data(
    original_df = cdc_survey_pmom_1plus_dep_screener,
    columns = gnb_model_features
)

rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=1,
    replacement=False
)
gb_x_train_rus, gb_y_train_rus = rus.fit_resample(gb_x_train,gb_y_train)

print(f"gb_x_train: {gb_x_train.shape}")
print(f"gb_y_train: {gb_y_train.shape}")
print(f"gb_x_test: {gb_x_test.shape}")
print(f"gb_y_test: {gb_y_test.shape}")
print(f"gb_x_train_rus: {gb_x_train_rus.shape}")

gb = GradientBoostingClassifier(random_state=42)
gb.fit(gb_x_train_rus, gb_y_train_rus)
#gb_opt9.fit(x_train, y_train)
gb_y_pred  = gb.predict(gb_x_test)
gb_score = get_performance_df(gb_y_test, gb_y_pred,'Gradient Boosting Classifier')  
gb_score

gb_x_train: (4512, 31)
gb_y_train: (4512,)
gb_x_test: (1129, 31)
gb_y_test: (1129,)
gb_x_train_rus: (1100, 31)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gradient Boosting Classifier,Depressed (No),0.959079,0.747012,0.839866,1004.0,0.746678
1,Gradient Boosting Classifier,Depressed (Yes),0.268012,0.744000,0.394068,125.0,0.746678
2,Gradient Boosting Classifier,Macro Avg,0.613545,0.745506,0.616967,1129.0,0.746678
3,Gradient Boosting Classifier,Weighted Avg,0.882566,0.746678,0.790508,1129.0,0.746678


## Test Hyperparameters

# RF Classifier (train & predict on cdc_survey_pmom_0dep_screener)

## Current Model

In [47]:
def create_bin_lookup(
        feature,
        n_bins,
        encode,
        strategy,
        df_to_use):
        # make a new column with _bin suffix
        new_column_name = feature + '_bin'

        # get non-null values per column
        feature_values = df_to_use[feature].dropna()

        # reshape to be 1 column
        feature_values = feature_values.to_numpy().reshape([feature_values.shape[0],1])

        # create bins using estimator
        est = KBinsDiscretizer(
            n_bins=n_bins,
            encode=encode, 
            strategy=strategy, 
            subsample=None
        )
        est.fit(feature_values)
        feature_values_bin = pd.DataFrame(est.transform(feature_values))

        # dataframe with binned values
        feature_values_bin.columns = [new_column_name]

        # get original
        feature_values = pd.DataFrame(feature_values)
        feature_values.columns = ['original']

        # merge bin & non-binned values together to make a lookup
        feature_values = feature_values.merge(feature_values_bin, left_index=True, right_index=True)
        feature_value_bin_lookup = feature_values.groupby(['original',new_column_name]).count().reset_index()

        return feature_value_bin_lookup, new_column_name
    
feature_value_bin_lookup, new_column_name = create_bin_lookup(
    'count_days_seen_doctor_12mo',
    n_bins=10,
    encode='ordinal',
    strategy='uniform',
    df_to_use=cdc_survey_pmom_0dep_screener
)  

# prevent creating a column if already exists
# happens if you run this block multiple times
if new_column_name in cdc_survey_pmom_0dep_screener.columns:
    cdc_survey_pmom_0dep_screener.drop(columns=new_column_name,inplace=True)

# add bin column in a way that doesn't drop nulls
cdc_survey_pmom_0dep_screener = cdc_survey_pmom_0dep_screener.merge(
    feature_value_bin_lookup, 
    left_on='count_days_seen_doctor_12mo', 
    right_on='original', 
    how = 'left'
)

# drop column called "original" as was only used to join
cdc_survey_pmom_0dep_screener.drop(columns=['original'], inplace=True)  


rf_model_features = [
    'count_days_seen_doctor_12mo_bin',
    'times_with_12plus_alc',
    'seen_mental_health_professional',
    'count_lost_10plus_pounds',
    'arthritis',
    'horomones_not_bc',
    'is_usa_born',
    'times_with_8plus_alc',
    'time_since_last_healthcare',
    'work_schedule'
]

In [48]:
rf_x_train, rf_x_test, rf_y_train, rf_y_test = get_model_data(
    original_df = cdc_survey_pmom_0dep_screener,
    columns = rf_model_features
)

rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=0.12,
    replacement=False
)


rf_x_train_rus, rf_y_train_rus = rus.fit_resample(rf_x_train,rf_y_train)

print(rf_x_train.shape)
print(rf_x_test.shape)
print(rf_y_train.shape)
print(rf_y_test.shape)
print(rf_x_train_rus.shape)
print(rf_y_train_rus.shape)

rf = RandomForestClassifier(random_state=42)
rf.fit(rf_x_train_rus, rf_y_train_rus)
rf_pred = rf.predict(rf_x_test)
rf_score = get_performance_df(rf_y_test, rf_pred,'Random Forest Classifier')
rf_score

(2677, 10)
(670, 10)
(2677,)
(670,)
(849, 10)
(849,)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Random Forest Classifier,Depressed (No),0.964615,0.979688,0.972093,640.0,0.946269
1,Random Forest Classifier,Depressed (Yes),0.350000,0.233333,0.280000,30.0,0.946269
2,Random Forest Classifier,Macro Avg,0.657308,0.606510,0.626047,670.0,0.946269
3,Random Forest Classifier,Weighted Avg,0.937095,0.946269,0.941104,670.0,0.946269


## Test Hyperparams

In [37]:
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# Number of trees in random forest
n_estimators = list(range(10,110, 10))
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2', None]
# Maximum number of levels in tree
max_depth = list(range(10,120, 10))
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

rf_param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_param_grid

{'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 'max_features': ['auto', 'sqrt', 'log2', None],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [ ]:
rf_grid_search = GridSearchCV(
    RandomForestClassifier(),
    param_grid=rf_param_grid
)
rf_grid_search.fit(rf_x_train_rus, rf_y_train_rus)

In [17]:
print(rf_grid_search.best_estimator_)

RandomForestClassifier(max_depth=3, max_leaf_nodes=9, n_estimators=25)


In [ ]:
print(rf_x_train_rus.shape)

rf_hyper = RandomForestClassifier(
    n_estimators=,
    max_features=,
    max_depth=,
    min_samples_split=,
    min_samples_leaf=,
    bootstrap=,
    random_state=42
)
rf_hyper.fit(rf_x_train_rus, rf_y_train_rus)
rf_hyper_pred = rf_hyper.predict(rf_x_test)
rf_hyper_score = get_performance_df(rf_y_test, rf_hyper_pred,'Random Forest Classifier')
rf_hyper_score

# Misc 

## GBC Classifier (train and predict on cdc_survey_pmom_0dep_screener)

## Current Model

In [45]:
gb_model_features = [
    'times_with_12plus_alc',
    'seen_mental_health_professional',
    'count_days_seen_doctor_12mo',
    'count_lost_10plus_pounds',
    'arthritis',
    'horomones_not_bc',
    'is_usa_born',
    'times_with_8plus_alc',
    'time_since_last_healthcare',
    'work_schedule',
    'age_in_years'
]

In [46]:
gb_x_train, gb_x_test, gb_y_train, gb_y_test = get_model_data(
    original_df = cdc_survey_pmom_0dep_screener,
    columns = gb_model_features
)

rus = RandomUnderSampler(
    random_state=42, 
    sampling_strategy=0.12,
    replacement=False
)

gb_x_train_rus, gb_y_train_rus = rus.fit_resample(gb_x_train,gb_y_train)

print(gb_x_train.shape)
print(gb_x_test.shape)
print(gb_y_train.shape)
print(gb_y_test.shape)
print(gb_x_train_rus.shape)
print(gb_y_train_rus.shape)

gb = GradientBoostingClassifier(random_state=42)
gb.fit(gb_x_train_rus, gb_y_train_rus)
gb_pred = gb.predict(gb_x_test)
gb_score = get_performance_df(gb_y_test, gb_pred,'Gradient Boosting Classifier')
gb_score

(2677, 11)
(670, 11)
(2677,)
(670,)
(849, 11)
(849,)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gradient Boosting Classifier,Depressed (No),0.967239,0.968750,0.967994,640.0,0.938806
1,Gradient Boosting Classifier,Depressed (Yes),0.310345,0.300000,0.305085,30.0,0.938806
2,Gradient Boosting Classifier,Macro Avg,0.638792,0.634375,0.636539,670.0,0.938806
3,Gradient Boosting Classifier,Weighted Avg,0.937826,0.938806,0.938311,670.0,0.938806


## Test Hyperparameters

In [20]:
# https://www.kaggle.com/code/hatone/gradientboostingclassifier-with-gridsearchcv/script

gb_param_grid = {
    "loss":["log_loss","exponential","deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3],
    #"min_samples_split": np.linspace(0.1, 0.5, 12),
    #"min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[2,3,5,8],
    "max_features":["log2","sqrt"],
    #"criterion": ["friedman_mse", "mae","squared_error"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10, 30, 70, 100, 200]
}
gb_param_grid

{'loss': ['log_loss', 'exponential', 'deviance'],
 'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3],
 'max_depth': [2, 3, 5, 8],
 'max_features': ['log2', 'sqrt'],
 'subsample': [0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
 'n_estimators': [10, 30, 70, 100, 200]}

In [21]:
gb_grid_search = GridSearchCV(
    GradientBoostingClassifier(),
    param_grid=gb_param_grid
)
gb_grid_search.fit(gb_x_train_rus, gb_y_train_rus)

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15,
                                           0.2, 0.3],
                         'loss': ['log_loss', 'exponential', 'deviance'],
                         'max_depth': [2, 3, 5, 8],
                         'max_features': ['log2', 'sqrt'],
                         'n_estimators': [10, 30, 70, 100, 200],
                         'subsample': [0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0]})

In [22]:
print(gb_grid_search.best_estimator_)

GradientBoostingClassifier(learning_rate=0.05, loss='exponential', max_depth=2,
                           max_features='log2', n_estimators=200,
                           subsample=0.8)


In [23]:
print(gb_x_train_rus.shape)

gb_hyper = GradientBoostingClassifier(
    learning_rate=0.05,
    loss='exponential',
    max_depth=2,
    max_features='log2',
    n_estimators=200,
    subsample=0.8,
    random_state=42
)
gb_hyper.fit(gb_x_train_rus, gb_y_train_rus)
gb_hyper_pred = gb_hyper.predict(gb_x_test)
gb_hyper_score = get_performance_df(gb_y_test, gb_hyper_pred,'Gradient Boosting Classifier (Hyper)')
gb_hyper_score

(849, 12)


,model,grouping,precision,recall,f1-score,support,accuracy
0,Gradient Boosting Classifier (Hyper),Depressed (No),0.961538,0.976562,0.968992,640.0,0.940299
1,Gradient Boosting Classifier (Hyper),Depressed (Yes),0.250000,0.166667,0.200000,30.0,0.940299
2,Gradient Boosting Classifier (Hyper),Macro Avg,0.605769,0.571615,0.584496,670.0,0.940299
3,Gradient Boosting Classifier (Hyper),Weighted Avg,0.929679,0.940299,0.934560,670.0,0.940299
